In [ ]:
# Ideas for linear regression. 
# Time vs. Collision Rates
# Multiple regressions on one plot based on mode or county.
# Mileage vs. Collision Rates
# Mutliple regressions on one based on injury outcome.
# Time vs. Injury Total Per Collision

#Import dependencies
from pathlib import Path
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt


 
#Open filtered CSV file
traffic_injuries = Path("Resources/road-traffic-injuries-2002-2010.csv")

df = pd.read_csv(traffic_injuries)

# Read in the data. 
df



In [ ]:
#clean data to remove null values and columns 
clean_df = df.drop(columns = ["ind_definition","version"])
clean_df = clean_df.drop(clean_df[clean_df['ind_id'] == "END OF TABLE"].index)
clean_df = clean_df.drop(columns = ["ind_id", 'LL95CI_avmtrate', 'UL95CI_avmtrate',
       'avmtrate_se', 'avmtrate_rse', 'CA_decile_avmt', 'CA_RR_avmtrate',
       'groupquarters','LL95CI_poprate', 'UL95CI_poprate',
       'poprate_se', 'poprate_rse', 'CA_decile_pop', 'CA_RR_poprate',
       'avmttotal', 'avmtrate'])
clean_df = clean_df[clean_df['severity'] != 'ND']
clean_df = clean_df[clean_df['mode'] != 'All modes']
clean_df = clean_df[~clean_df['reportyear'].isin(['2002-2004','2005-2007','2008-2010','2006-2010'])]

clean_df

In [ ]:
# Create a Bar chart- Accidents by Region and severity
yearwise_severity = clean_df.groupby(['region_name', 'severity']).size().reset_index(name='Accident Count')
pivot_data = yearwise_severity.pivot_table(index='region_name', columns='severity', values='Accident Count').sort_index(axis=1)
pivot_data.plot.bar(figsize=(12, 6))
plt.xlabel('Region')
plt.ylabel('Accident Count')
plt.ylim(0, 15000)
plt.title('Accidents by region_name and Severity')
plt.xticks(rotation=45)
plt.show()